In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import sys
import tensorflow as tf
#import segmentation_models as sm
from keras import backend as K
K.set_image_data_format('channels_last')
from ImageDataAugmentor.image_data_augmentor import *
import albumentations
from datetime import datetime
tf.__version__

'2.3.0'

In [2]:
!lscpu | grep "Model name"

Model name:                      Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz


In [3]:
def image_preprocessing(x):
    x = x/255. # rescale to [0,1]
    return(x)
    
TEST_AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5)
])

img_test_data_gen = ImageDataAugmentor(augment=TEST_AUGMENTATIONS, 
                                  augment_seed=123,
                                  preprocess_input = image_preprocessing)
img_test_gen = img_test_data_gen.flow_from_directory('./test_data/img', 
                                           class_mode=None, 
                                           shuffle=True, 
                                           seed=123, 
                                           color_mode='rgb', 
                                           target_size=(512, 512),
                                           batch_size=4)

Found 6812 images belonging to 1 classes.


In [4]:
def query(model):
    image_batch = next(img_test_gen)
    for i in range(4):
        start = datetime.now()
        pred = model.predict_on_batch(np.expand_dims(image_batch[i,:,:,:], axis=0))
        end = datetime.now()
        print(end - start)

# Test performance on image 512*512

## U-Net (resnet18)

In [7]:
model = sm.Unet(backbone_name='resnet18',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)
    
model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

0:00:01.529350
0:00:00.477358
0:00:00.515232
0:00:00.518919


## U-Net (resnet34)

In [8]:
model = sm.Unet(backbone_name='resnet34',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)

model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

0:00:02.173203
0:00:00.558949
0:00:00.599785
0:00:00.569993


## U-Net (resnet50)

In [9]:
model = sm.Unet(backbone_name='resnet50',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)

model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

0:00:03.279154
0:00:00.973155
0:00:00.967939
0:00:00.890561


## U-Net (efn-B0)

In [10]:
model = sm.Unet(backbone_name='efficientnetb0',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)

model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

0:00:03.942730
0:00:00.778575
0:00:00.721337
0:00:00.747951


## U-Net (efn-B2)

In [11]:
model = sm.Unet(backbone_name='efficientnetb2',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)

model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

0:00:05.713695
0:00:00.978065
0:00:00.956792
0:00:00.970785


## U-Net (mobilenet-v1)

In [12]:
model = sm.Unet(backbone_name='mobilenet',
    classes=1,
    encoder_weights='imagenet',
    input_shape=(512, 512, 3),
    encoder_freeze=True)

model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
query(model)

/home/gosha20777/anaconda3/envs/tf-1-14/lib/python3.7/site-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


0:00:03.919622
0:00:00.656635
0:00:00.602401
0:00:00.575693


## U-Net (Custom-1)

In [5]:
from keras import Model
from keras.layers import Input, Dense, Concatenate, Convolution2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, Callback, ModelCheckpoint
from keras.utils import Sequence
from keras.regularizers import l2

def model1(img_rows, img_cols, optimizer, reg=0.1):
    '''https://github.com/yihui-he/u-net'''
    inputs = Input(shape=(img_rows, img_cols, 3))
    bn1 = BatchNormalization()(inputs)
    conv1 = Convolution2D(32, (3, 3), activation='relu', padding='same')(bn1)
    conv1 = Convolution2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Convolution2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Convolution2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Convolution2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Convolution2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Convolution2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = Convolution2D(512, (3, 3), activation='elu', padding='same')(pool4)
    conv5 = Convolution2D(512, (3, 3), activation='elu', padding='same')(conv5)
    up6 = Concatenate()([Convolution2D(256, (2, 2), activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv5)), conv4])
    conv6 = Convolution2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Convolution2D(256, (3, 3), activation='relu', padding='same')(conv6)
    up7 = Concatenate()([Convolution2D(128, (2, 2),activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6)), conv3])
    conv7 = Convolution2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Convolution2D(128, (3, 3), activation='relu', padding='same')(conv7)
    up8 = Concatenate()([Convolution2D(64, (2, 2),activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7)), conv2])
    conv8 = Convolution2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Convolution2D(64, (3, 3), activation='relu', padding='same')(conv8)
    up9 = Concatenate()([Convolution2D(32, (2, 2),activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8)), conv1])
    conv9 = Convolution2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Convolution2D(32, (3, 3), activation='relu', padding='same')(conv9)
    bn = BatchNormalization()(conv9)    
    conv10 = Convolution2D(1, (1, 1), activation='sigmoid')(bn)
    model = Model(inputs=inputs, outputs=conv10)
    for layer in range(len(model.layers)):
        model.layers[layer].kernel_regularizer=l2(reg)
        model.layers[layer].bias_regularizer=l2(reg)
    
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

    print('U-Net compiled,  batch_norm1=True, batch_norm10=True, 4 skips.')
    print('Input shape:', model.input_shape)
    print('Output shape:', model.output_shape)
    return model

In [6]:
model = model1(512, 512, optimizer=Adam(learning_rate=1e-3))
query(model)

U-Net compiled,  batch_norm1=True, batch_norm10=True, 4 skips.
Input shape: (None, 512, 512, 3)
Output shape: (None, 512, 512, 1)
0:00:00.471553
0:00:00.318344
0:00:00.323204
0:00:00.326768


### Keras-Retinanet (resnet-50)

In [7]:
from keras_retinanet import models

def create_model(backbone_name, num_classes=1):
    backbone_factory = models.backbone(backbone_name)
    model = backbone_factory.retinanet(num_classes)
    return models.convert_model(model)

backbone = 'resnet50'
model = create_model(backbone)
query(model)

0:00:01.348107
0:00:00.281428
0:00:00.280437
0:00:00.280389


In [9]:
#backbone = 'EfficientNetB0'
#model = create_model(backbone)
#query(model)

### Keras-Retinanet (mobilenet)

In [11]:
#backbone = 'mobilenet224_0.1'
#model = create_model(backbone)
#query(model)

### YOLO 4 (416*416)

In [12]:
from yolo_4.models import Yolov4
yolo = Yolov4(class_name_path = 'yolo_4/class_names/coco_classes.txt',
               weight_path=None)

print(yolo.img_size)
model = yolo.inference_model

def image_preprocessing(x):
    x = x/255. # rescale to [0,1]
    return(x)
    
TEST_AUGMENTATIONS = albumentations.Compose([
    albumentations.HorizontalFlip(p=0.5)
])

yolo_img_test_data_gen = ImageDataAugmentor(augment=TEST_AUGMENTATIONS, 
                                  augment_seed=123,
                                  preprocess_input = image_preprocessing)
yolo_img_test_gen = yolo_img_test_data_gen.flow_from_directory('./test_data/img', 
                                           class_mode=None, 
                                           shuffle=True, 
                                           seed=123, 
                                           color_mode='rgb', 
                                           target_size=(416, 416),
                                           batch_size=4)

def yolo_query(model):
    image_batch = next(yolo_img_test_gen)
    for i in range(4):
        start = datetime.now()
        pred = model.predict_on_batch(np.expand_dims(image_batch[i,:,:,:], axis=0))
        end = datetime.now()
        print(end - start)

yolo_query(model)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
nms iou: 0.413 score: 0.3
(416, 416, 3)
Found 6812 images belonging to 1 classes.
0:00:01.488943
0:00:00.285894
0:00:00.287846
0:00:00.278481


### YOLO 4-tf (416*416)

In [15]:
!cd ../tensorflow-yolov4-tflite && \
   python detect.py --weights ./checkpoints/yolov4-416 \
   --size 416 --model yolov4 \
   --image ../lacmus-research/test_data/img/1/0___n04759.jpg

0:00:00.658849
0:00:00.255762
0:00:00.254708
0:00:00.265157


### YOLO 4-tf-tiny (416*416)

In [17]:
!cd ../tensorflow-yolov4-tflite && \
   python detect.py --weights ./checkpoints/yolov4-tiny-416 \
   --size 416 --model yolov4 \
   --image ../lacmus-research/test_data/img/1/0___n04759.jpg --tiny

0:00:00.142459
0:00:00.025333
0:00:00.028778
0:00:00.026272
